In [36]:
###   Importação de Bibliotecas   ###

In [37]:
import pandas as pd
import numpy as np
import re
import datetime

In [38]:
###   Chamando Dados   ###

In [39]:
### Base com cálculos sobre os supercondutores ###

supercon = pd.read_csv(r'C:\Users\muril\Desktop\superconduct\TCC\dados_supercon_murilo\base_supercondutores.csv')

### Tiro a indexação do csv ###   
supercon=supercon.drop(columns=['Unnamed: 0'])
    
###  Base com distribuição atômica dos elementos do material supercondutor ###

elementos = pd.read_csv(r'C:\Users\muril\Desktop\superconduct\TCC\dados_supercon_murilo\elementos_atomos.csv')

###  Tiro a indexação do csv ###
elementos=elementos.drop(columns=['Unnamed: 0'])
    
### Base propriedades dos elementos ###

propriedades = pd.read_csv(r'C:\Users\muril\Desktop\superconduct\TCC\dados_supercon_murilo\sub_elementos_dados.csv')

### Coloco os nome dos elementos como indexação ###
propriedades = propriedades.set_index("Element")

### Base com novos supercondutores ###
novos = pd.read_csv(r'C:\Users\muril\Desktop\superconduct\TCC\dados_supercon_murilo\novos_supercondutores.csv', sep=';')

### Removo possíveis valores duplicados ###
novos = novos.drop_duplicates().reset_index(drop=True)

In [40]:
### A função a seguir, limpa valores duplicados e garante que que os registros novos nunca foram expostos a testes ###

In [41]:
def verificador_novos():
    
    novos_novos = novos

    novos_dados=np.array(novos_novos)
    antigos_dados = np.array(elementos.iloc[:,[-1,-2]])

    lista_novos = []

    for i in novos_dados:

        j = i[0] + '-'+str(int(i[1]))

        lista_novos.append(j)

    

    lista_antigos = []

    for i in antigos_dados:

        j = i[0] +'-'+ str(int(i[1]))

        lista_antigos.append(j)

    

    indice = 0

    for i in lista_novos:

        if i in lista_antigos:

            novos_novos = novos_novos.drop(indice)

        indice += 1


    novos_novos = novos_novos.reset_index(drop=True)
    
    return novos_novos

In [42]:
###   Função que calcula parâmetros para serem inseridos no DataFrame supercon  ###
### Função retorna um DataFrame que pode ser usado para predizer Tc ou inserir registros no DataFrame supercon ###

In [43]:
def calcular(v):
    
    argumentos = v
    
    ### respondendo qual o material está sendo analisado ###
    
    material = ""
    
    for concatenar in argumentos:
        
        material += str(concatenar)
        
    print("Material: ",material)
    
    ### Elementos e suas distribuições ###
    
    
    num_elementos = int(len(argumentos)/2)
    
    num_atomos = 0
    
    for atomo in range(num_elementos):
        
        num_atomos += float(argumentos[(2*atomo + 1)])
        
    
    ### Criação de um vetor preditivo ###
    
    v_preditivo = []
    
    v_preditivo.append(num_elementos)
    
    ### Elementos e suas distribuições: fator p ###
    
    fator_p = []
    
    for p in range(num_elementos):
        
        fator_p.append(argumentos[(2*p + 1)]/num_atomos)
    
    ### Elementos e suas distribuições para cada propriedade ###
    
    colunas = propriedades.columns[1:len(propriedades.columns)]
    
    for icolunas in colunas:
    

         ### Elementos e suas distribuições: fator w (Calculado sobre as propriedades t) ###

        t = []
        fator_w = []

        for ipropriedade in range(num_elementos):

            t.append(propriedades.loc[argumentos[2*ipropriedade],icolunas])
            

        for ifator_w in range(num_elementos):

            fator_w.append(t[ifator_w]/sum(t))

         ### Elementos e suas distribuições: Coeficiente Geral (Calculado sobre os coeficientes p e w) ###

        coef_gerais = []

        divisor = np.sum(np.array(fator_p)*np.array(fator_w))

        for icoef in range(num_elementos):

            coef_gerais.append((fator_p[icoef]*fator_w[icoef])/divisor)


        ### Características ###

        media = (np.mean(np.array(t)))
        
        v_preditivo.append(media)

        media_ponderada = np.sum(np.array(t)*np.array(fator_p))
        
        v_preditivo.append(media_ponderada)

        media_geometrica = np.prod(t)**(1/len(t))
            
        v_preditivo.append(media_geometrica)

        media_geo_ponderada = np.prod(np.array(t)**np.array(fator_p))
        
        v_preditivo.append(media_geo_ponderada)

        entropia = np.sum(-1*np.array(fator_w)*np.log(np.array(fator_w)))
        
        v_preditivo.append(entropia)

        entropia_peso = np.sum(-1*np.array(coef_gerais)*np.log(np.array(coef_gerais)))

        v_preditivo.append(entropia_peso)

        Range = max(t) - min(t)
        
        v_preditivo.append(Range)

        Range_ponderado = np.amax(np.array(t)*np.array(fator_p))-np.amin(np.array(t)*np.array(fator_p))
        
        v_preditivo.append(Range_ponderado)

        desv_padrao  =  np.sqrt((np.sum((np.array(t)-media)**2)/len(t)))
        
        v_preditivo.append(desv_padrao)

        desv_padrao_ponderado = np.sqrt(np.sum(np.array(fator_p)*((np.array(t)-media_ponderada)**2)))
        
        v_preditivo.append(desv_padrao_ponderado)
        
    
    return pd.DataFrame(np.reshape(np.array(v_preditivo),(1, len(v_preditivo))), columns=supercon.columns[0:-1])
    

In [44]:
supercon

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4.0,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.906070,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1.0,1.085714,0.433013,0.437059,29.00
1,5.0,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.906070,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2.0,1.128571,0.632456,0.468606,26.00
2,4.0,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.906070,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1.0,1.114286,0.433013,0.444697,19.00
3,4.0,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.906070,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1.0,1.100000,0.433013,0.440952,22.00
4,4.0,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.906070,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1.0,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21534,2.0,97.325500,44.085808,44.581322,18.642955,0.214516,0.497712,173.029000,26.621885,86.514500,...,3.576923,4.242641,3.427770,0.636514,0.628799,3.0,1.269231,1.500000,1.182340,2.10
21535,2.0,129.953940,114.289135,119.530936,105.235852,0.614050,0.688446,101.987880,11.073449,50.993940,...,5.653595,5.477226,5.632776,0.689009,0.616156,1.0,2.189542,0.500000,0.475824,3.20
21536,2.0,129.953940,114.721984,119.530936,105.606884,0.614050,0.687504,101.987880,12.176529,50.993940,...,5.649351,5.477226,5.628419,0.689009,0.619376,1.0,2.142857,0.500000,0.477173,2.95
21537,2.0,129.953940,115.149248,119.530936,105.974410,0.614050,0.686497,101.987880,13.265377,50.993940,...,5.645161,5.477226,5.624121,0.689009,0.622487,1.0,2.096774,0.500000,0.478464,2.10


In [45]:
### Função para distribuição dos elementos no formato do DataFrame elementos   ###
### Retorna um DataFrame que pode ser usado para inserir valores no DataFrame elementos ###

In [46]:
def distribuir(Tc,v):
     
    ### Vetores com atomos e numero de atomos ###
    
    atomo = []
    num_atomo = []
    
    for i in range(len(v)):
        
        if i%2 == 0:
            
            atomo.append(v[i])
            
        else:
            
            num_atomo.append(v[i])
             
    ### Material ###
    
    material = ""
    
    for concatenar in v:
        
        material += str(concatenar)
    
    #### Criação de nova linha para o DataFrame elemento ###
    
    ### Colunas a serem preenchidas ###
    
    vetor_de_atribuicao_colunas = atomo
    
    for add in ['critical_temp','material']:
        
        vetor_de_atribuicao_colunas.append(add)
    
    ### Valores das colunas ###
    
    vetor_de_atribuicao_valores = num_atomo
    
    for add2 in [Tc,material]:
        
        vetor_de_atribuicao_valores.append(add2)
    
    ### Geração de um DataFrame para ser incorporado ao DataFrame elementos ###
    
    frame_zeros = pd.DataFrame(np.zeros((1,len(elementos.columns))), columns=elementos.columns)
    
    frame_zeros.loc[:,vetor_de_atribuicao_colunas] = vetor_de_atribuicao_valores
    
    return frame_zeros      
    

In [47]:
###   Função para transformar um texto contendo uma formula química em um vetor   ###

In [48]:
def fatiar(v):
    
    ### Converte o texto em lista ###
    
    texto=re.findall('[A-Z][a-z]?|[0.-9.]+', v)       
    
    ### Transforma string em número ###
    
    for i in range(len(texto)):
        
        try:            
            
            texto[i] = float(texto[i])
        
        except:
                       
            texto[i] = texto[i]
            
    
    ### Corrigir número de átomos ###
    
    
    i1 = 0
    
    while (i1 <= (len(texto)-1)):
        
        if type(texto[i1]) == str and i1 != (len(texto) - 1):
            
            if type(texto[(i1+1)]) != str:
                
                texto[i1] = texto[i1]
                
            elif type(texto[(i1+1)]) == str:
                
                texto.insert((i1+1),1)
                
        
        if type(texto[i1]) == float and i1 != (len(texto)-1):
            
            if type(texto[(i1+1)]) != float:
                
                texto[i1] = texto[i1]
                
            elif type(texto[(i1+1)]) == float:
                
                texto[i1] = float(str(texto[i1]) + '.' + str(texto[(i1+1)]))
                
                del(texto[(i1+1)])
           
        
        if i1 == (len(texto)-1) and type(texto[i1]) == str:
            
            texto.append(1)
           
        i1+=1
        
        
    for i in range(len(texto)):
        
        if type(texto[i]) == float and texto[i]%1 == 0:
            
            
            texto[i] = int(texto[i])           
          
    
    return texto    
    

In [49]:
###   Geração de um novo Frame elementos, adiciona os supercondutores do Frame novos   ###

In [50]:
def novo_elementos():
   
    novos1 = verificador_novos()
    
    
    c = 0

    nova_distribuicao_elementos = pd.DataFrame(elementos)

    for i in novos1.iloc[:,0]:

        if 'Th' not in fatiar(i) and 'M' not in fatiar(i) and 'U' not in fatiar(i) and 'D' not in fatiar(i):

            nova_distribuicao_elementos = nova_distribuicao_elementos.append(distribuir(novos1.iloc[c,1],fatiar(i)), ignore_index=True)

        c+=1
        
    return nova_distribuicao_elementos

In [51]:
###   Visualiza os novos dados incorporados ao Frame elementos   ###

In [52]:
#visualizar_novos_dados_frame_elementos = novo_elementos()
#entrada_dados_elementos = visualizar_novos_dados_frame_elementos.iloc[elementos.shape[0]-1:,]
#entrada_dados_elementos

In [53]:
###   Geração de um novo Frame supercon, adiciona os supercondutores do Frame novos   ###

In [54]:
def novo_supercon():

    novos = verificador_novos()
    
    c = 0

    novo_supercon = pd.DataFrame(supercon)

    for i in novos.iloc[:,0]:

        if 'Th' not in fatiar(i) and 'M' not in fatiar(i) and 'U' not in fatiar(i) and 'D' not in fatiar(i):

                       
            resultado_calculo = calcular(fatiar(i))          
            
            frame_supercon = pd.DataFrame(columns=supercon.columns)
            
            frame_supercon[resultado_calculo.columns] = resultado_calculo
            frame_supercon[supercon.columns[len(supercon.columns)-1]] = novos.iloc[c,1]
            
            
            novo_supercon = novo_supercon.append(frame_supercon, ignore_index=True)

        c+=1
        
    return novo_supercon

In [55]:
novo_supercon()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4.0,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.906070,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1.0,1.085714,0.433013,0.437059,29.00
1,5.0,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.906070,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2.0,1.128571,0.632456,0.468606,26.00
2,4.0,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.906070,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1.0,1.114286,0.433013,0.444697,19.00
3,4.0,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.906070,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1.0,1.100000,0.433013,0.440952,22.00
4,4.0,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.906070,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1.0,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21534,2.0,97.325500,44.085808,44.581322,18.642955,0.214516,0.497712,173.029000,26.621885,86.514500,...,3.576923,4.242641,3.427770,0.636514,0.628799,3.0,1.269231,1.500000,1.182340,2.10
21535,2.0,129.953940,114.289135,119.530936,105.235852,0.614050,0.688446,101.987880,11.073449,50.993940,...,5.653595,5.477226,5.632776,0.689009,0.616156,1.0,2.189542,0.500000,0.475824,3.20
21536,2.0,129.953940,114.721984,119.530936,105.606884,0.614050,0.687504,101.987880,12.176529,50.993940,...,5.649351,5.477226,5.628419,0.689009,0.619376,1.0,2.142857,0.500000,0.477173,2.95
21537,2.0,129.953940,115.149248,119.530936,105.974410,0.614050,0.686497,101.987880,13.265377,50.993940,...,5.645161,5.477226,5.624121,0.689009,0.622487,1.0,2.096774,0.500000,0.478464,2.10


In [56]:
###   Visualiza os novos dados incorporados ao Frame supercon   ###

In [57]:
#visualizar_novos_dados_frame_supercon = novo_supercon()
#entrada_dados_supercon = visualizar_novos_dados_frame_supercon.iloc[supercon.shape[0]-1:,]
#entrada_dados_supercon

In [58]:
###   Salvar NOVOS conjuntos de Dados   ###

In [59]:
def salvar():

    ### Caminho das pastas ###

    caminho_base = r'C:\Users\muril\Desktop\superconduct\TCC\dados_supercon_murilo'
    caminho_repositorio = r'C:\Users\muril\Desktop\superconduct\TCC\dados_supercon_murilo\repositorio_dados'

    ### Base Supercondutores ###

    supercon_name = r'\base_supercondutores'+'.csv'
    supercon_name_data = r'\base_supercondutores_V'+datetime.datetime.now().strftime('%Y%d%m_%H%M')+'.csv'

    ### Repositório ###
    supercon.to_csv(caminho_repositorio+supercon_name_data)
    ### Base Atual ###
    Base_Atual = novo_supercon()
    Base_Atual.to_csv(caminho_base+supercon_name)

    ### Base Elementos ###

    elemento_nome = r'\elementos_atomos'+'.csv'
    elemento_nome_data = r'\elementos_atomos_V'+datetime.datetime.now().strftime('%Y%d%m_%H%M')+'.csv'

    ### Repositório ###
    elementos.to_csv(caminho_repositorio+elemento_nome_data)
    ### Base Atual ###
    Base_Atual = novo_elementos()
    Base_Atual.to_csv(caminho_base+elemento_nome)

In [60]:
#salvar()

In [61]:
###   Base contem todos os dados a serem explorados nos modelos de ML   ###   

In [62]:
base_geral = supercon.copy()
base_geral['material'] = elementos['material']
base_geral

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp,material
0,4.0,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.906070,31.794921,51.968828,...,2.213364,2.219783,1.368922,1.066221,1.0,1.085714,0.433013,0.437059,29.00,Ba0.2La1.8Cu1O4
1,5.0,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.906070,36.161939,47.094633,...,1.888175,2.210679,1.557113,1.047221,2.0,1.128571,0.632456,0.468606,26.00,Ba0.1La1.9Ag0.1Cu0.9O4
2,4.0,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.906070,35.741099,51.968828,...,2.213364,2.232679,1.368922,1.029175,1.0,1.114286,0.433013,0.444697,19.00,Ba0.1La1.9Cu1O4
3,4.0,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.906070,33.768010,51.968828,...,2.213364,2.226222,1.368922,1.048834,1.0,1.100000,0.433013,0.440952,22.00,Ba0.15La1.85Cu1O4
4,4.0,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.906070,27.848743,51.968828,...,2.213364,2.206963,1.368922,1.096052,1.0,1.057143,0.433013,0.428809,23.00,Ba0.3La1.7Cu1O4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21534,2.0,97.325500,44.085808,44.581322,18.642955,0.214516,0.497712,173.029000,26.621885,86.514500,...,4.242641,3.427770,0.636514,0.628799,3.0,1.269231,1.500000,1.182340,2.10,B4.2W1
21535,2.0,129.953940,114.289135,119.530936,105.235852,0.614050,0.688446,101.987880,11.073449,50.993940,...,5.477226,5.632776,0.689009,0.616156,1.0,2.189542,0.500000,0.475824,3.20,Se2Ta1.06
21536,2.0,129.953940,114.721984,119.530936,105.606884,0.614050,0.687504,101.987880,12.176529,50.993940,...,5.477226,5.628419,0.689009,0.619376,1.0,2.142857,0.500000,0.477173,2.95,Se2Ta1.08
21537,2.0,129.953940,115.149248,119.530936,105.974410,0.614050,0.686497,101.987880,13.265377,50.993940,...,5.477226,5.624121,0.689009,0.622487,1.0,2.096774,0.500000,0.478464,2.10,Se2Ta1.1


In [63]:
supercon

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4.0,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.906070,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1.0,1.085714,0.433013,0.437059,29.00
1,5.0,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.906070,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2.0,1.128571,0.632456,0.468606,26.00
2,4.0,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.906070,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1.0,1.114286,0.433013,0.444697,19.00
3,4.0,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.906070,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1.0,1.100000,0.433013,0.440952,22.00
4,4.0,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.906070,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1.0,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21534,2.0,97.325500,44.085808,44.581322,18.642955,0.214516,0.497712,173.029000,26.621885,86.514500,...,3.576923,4.242641,3.427770,0.636514,0.628799,3.0,1.269231,1.500000,1.182340,2.10
21535,2.0,129.953940,114.289135,119.530936,105.235852,0.614050,0.688446,101.987880,11.073449,50.993940,...,5.653595,5.477226,5.632776,0.689009,0.616156,1.0,2.189542,0.500000,0.475824,3.20
21536,2.0,129.953940,114.721984,119.530936,105.606884,0.614050,0.687504,101.987880,12.176529,50.993940,...,5.649351,5.477226,5.628419,0.689009,0.619376,1.0,2.142857,0.500000,0.477173,2.95
21537,2.0,129.953940,115.149248,119.530936,105.974410,0.614050,0.686497,101.987880,13.265377,50.993940,...,5.645161,5.477226,5.624121,0.689009,0.622487,1.0,2.096774,0.500000,0.478464,2.10


In [64]:
###   Conjunto X   ###

In [65]:
X_modelagem = base_geral.iloc[:,:base_geral.shape[1]-2]
X_modelagem.describe()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
count,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,...,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000,21539.000000
mean,4.110590,87.656529,73.158278,71.407160,58.727590,1.164685,1.062467,115.552879,33.312449,44.395364,...,3.208011,3.165130,3.065292,3.066781,1.294313,1.051560,2.050467,1.488467,0.842723,0.677858
std,1.439764,29.748893,33.563601,31.125815,36.718826,0.365162,0.400835,54.534660,26.967319,20.017244,...,1.048631,1.194940,1.050941,1.178835,0.392861,0.379702,1.246448,0.978602,0.485505,0.457329
min,1.000000,6.941000,6.423452,5.320573,1.960849,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,72.423073,52.178103,57.954438,35.260102,0.963421,0.773839,78.632800,16.877500,32.890369,...,2.333333,2.121429,2.289428,2.092686,1.060857,0.773468,1.000000,0.923077,0.471405,0.308800
50%,4.000000,85.097080,60.931731,66.361592,40.098709,1.199541,1.144041,122.906070,26.721863,45.123500,...,2.833333,2.666667,2.615321,2.449490,1.368922,1.156461,2.000000,1.064220,0.800000,0.500000
75%,5.000000,100.520500,86.302367,78.241067,73.336513,1.444537,1.357986,154.119320,38.505828,59.224069,...,4.000000,4.054000,3.779763,3.942875,1.589027,1.329399,3.000000,1.924800,1.213352,1.021409
max,9.000000,208.980400,208.980400,208.980400,208.980400,1.983797,1.958203,207.972460,205.589910,101.019700,...,7.000000,7.000000,7.000000,7.000000,2.141963,1.949739,6.000000,6.992200,3.000000,3.000000


In [66]:
###   Conjunto Y   ###

In [67]:
Y = base_geral.iloc[:,[-2,-1]]

#Conjuto Y usado na modelagem
Y_modelagem = base_geral.iloc[:,-2]

ver_contagem = pd.DataFrame(Y['material'].value_counts())
print(Y.describe())
print(ver_contagem.describe())

       critical_temp
count   21539.000000
mean       34.124146
std        34.191957
min         0.000210
25%         5.240000
50%        19.700000
75%        62.000000
max       185.000000
           material
count  15767.000000
mean       1.366081
std        1.943464
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max      110.000000


In [68]:
###   Função de relação entre composto e sua temperatura crítica (traz a média da temperatura crítica de determinado composto)   ###

In [69]:
def procura(procura):
    
    proc_mean =Y.loc[Y['material']==procura]
    x=proc_mean.median()
    print(x[0])
    
    return x[0]

In [70]:
calcular(fatiar('MgB2'))

Material:  Mg1B2


,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
0,2.0,17.558,15.309,16.209915,14.162583,0.617382,0.69144,13.494,0.894333,6.747,...,2.5,2.666667,2.44949,2.620741,0.673012,0.562335,1.0,1.333333,0.5,0.471405


In [71]:
fatiar('Ba0.2La1.8Cu1O4')

['Ba', 0.2, 'La', 1.8, 'Cu', 1, 'O', 4]